# Importing data and preparing them

In [1]:
import pandas as pd

def load_and_prepare_data():
    # Load and prepare the primary dataframe `df`
    df = pd.read_csv("V01_PIA_MONTHLY_BOM.csv", low_memory=False)
    
    # Data type corrections and filtering for `df`
    df['OUTPUT_MATERIAL_ID'] = df['OUTPUT_MATERIAL_ID'].astype(str)
    df['MONTHLY_DATE'] = pd.to_datetime(df['MONTHLY_DATE'])
    df = df[df['TOTAL_OUTPUT_QTY'] != 0].reset_index(drop=True)  # Remove rows with TOTAL_OUTPUT_QTY == 0
    df = df[df['MONTHLY_DATE'].dt.year == 2023].reset_index(drop=True)  # Filter for year 2022

    # Selecting the required columns
    selected_columns = df[
        ['MONTHLY_DATE', 'PLANT_ID', 'COMPONENT_ID', 'TOTAL_INPUT_QTY',
         'FINAL_COMPONENT_UOM', 'TOTAL_INPUT_MASS', 'OUTPUT_MATERIAL_ID',
         'TOTAL_OUTPUT_QTY', 'TOTAL_OUTPUT_MASS', 'FINAL_OUTPUT_MATERIAL_UOM','COMPONENT_DESC','COMPONENT_MATERIAL_TYPE',
         'COMPONENT_MASS_CONV_FACTOR', 'COMPONENT_MASS_UNIT', 'OUTPUT_MASS_CONV_FACTOR',	'OUTPUT_MASS_UNIT',	'OUTPUT_MATERIAL_DESC',	'OUTPUT_MATERIAL_TYPE']
    ]

    # Apply grouping functionality
    df = selected_columns.groupby(
        ['COMPONENT_ID', 'OUTPUT_MATERIAL_ID'], as_index=False, sort=False
    ).agg({
        'TOTAL_INPUT_QTY': 'sum',       # Sum the 'TOTAL_INPUT_QTY' column
        'TOTAL_OUTPUT_QTY': 'sum',      # Sum the 'TOTAL_OUTPUT_QTY' column
        'FINAL_COMPONENT_UOM': 'first', # Keep the first 'FINAL_COMPONENT_UOM' value
        'PLANT_ID': 'first',            # Keep the first 'PLANT_ID' value
        'FINAL_COMPONENT_UOM':'first',# Add more aggregation functions as needed
        'FINAL_OUTPUT_MATERIAL_UOM':'first',
        'COMPONENT_MATERIAL_TYPE': 'first',
        'COMPONENT_DESC':'first',
        'COMPONENT_MASS_CONV_FACTOR':'first',
        'COMPONENT_MASS_UNIT': 'first',
        # the following stuff is for level 0 item
        'OUTPUT_MASS_CONV_FACTOR':'first',
        'OUTPUT_MASS_UNIT':'first',
        'OUTPUT_MATERIAL_DESC':'first',
        'OUTPUT_MATERIAL_TYPE':'first'
        
    })

    # Load other datasets without additional processing
    plant = pd.read_csv('3M data/V01_PLANT.csv')
    plant_loc = pd.read_csv('3M data/V01_PIA_SUPPLIER_LOCATION_5-12-24.csv', low_memory=False)
    
    # Load and prepare `CDMS_DATA`
    CDMS_DATA = pd.read_csv("3M data/V01_CDMS_DATA.csv")
    CDMS_DATA['SAP_ID'] = CDMS_DATA['SAP_ID'].astype(str).str.replace(r'\.0$', '', regex=True)
    CDMS_DATA['CAS_'] = CDMS_DATA['CAS_'].fillna("")
    return df, plant, plant_loc, CDMS_DATA


# building adj_list

In [2]:
import time

def create_adj_list(df):
    # Filter data by the specified year, the following step is not needed since it happens when importing the data
    # filtered_data = df[df['MONTHLY_DATE'].dt.year == int(year)]
    
    adj_full = {}
    start_time = time.time()
    
    # Iterate through each row in the filtered data
    for row in df.itertuples():
        output_material_id = row.OUTPUT_MATERIAL_ID
        component_id = row.COMPONENT_ID
        row_index = row.Index  # Row index in filtered data

        # Add component_id and row index to adj_full dictionary
        if output_material_id in adj_full:
            adj_full[output_material_id].append((component_id, row_index))
        else:
            adj_full[output_material_id] = [(component_id, row_index)]

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")

    return adj_full


# creating a BOM dataframe

In [4]:
# test dfs

color = {}

def dfs(component_id, adj_full, ans, level):

    color[component_id] = 1 # marking the node as grey - in progress or being processed 
    
    for child, index in adj_full.get(component_id, []):# Retrieve each child and its index from adj_full

        if color.get(child) != 1:
            
            ans.append((child, index, level))  # Add each child as a tuple (child, index, level)
            dfs(child, adj_full, ans, level + 1)  # Recursively call dfs on each child and increase the level


    color[component_id] = 2 # marking the node as black - processed

In [3]:
def dfs(component_id, adj_full, ans, level):
    """
    Perform depth-first search to traverse the tree and collect descendants
    with their corresponding index and level.

    :param component_id: The current node (component) being processed.
    :param adj_full: The adjacency list (dict) containing the graph of components.
    :param ans: The list to store the result tuples (child, index, level).
    :param level: The current depth level in the tree.
    """
    for child, index in adj_full.get(component_id, []):  # Retrieve each child and its index from adj_full
        ans.append((child, index, level))  # Add each child as a tuple (child, index, level)
        dfs(child, adj_full, ans, level + 1)  # Recursively call dfs on each child and increase the level


def build_bom_dataframe(component_id, adj_full, df):
    """
    Build the BOM dataframe based on the component's tree structure from the adjacency list.

    :param component_id: The starting component ID to initiate the DFS.
    :param adj_full: The adjacency list (dict) containing the graph of components.
    :param df: The original dataframe containing the component details.
    :return: A new dataframe containing the descendants of the component with their corresponding levels.
    """
    ans = []  # Initialize the list to store (child, index, level) tuples
    
    # Perform DFS starting from the given component_id
    dfs(component_id, adj_full, ans, level=1)
    
    # Extract the indexes and levels from the DFS result
    indexes = [t[1] for t in ans]  # Extract all the second elements (indexes)
    levels = [t[2] for t in ans]   # Extract all the third elements (levels)

    # Select the rows from the DataFrame using the extracted indexes
    new_dataframe = df.iloc[indexes].copy()

    # Add the 'Level' column to the new dataframe
    new_dataframe['Level'] = levels
    new_new_dataframe = new_dataframe.reset_index(drop=True)
    return new_new_dataframe


# Preparing for the scalling 

In [5]:
import pandas as pd

def group_and_aggregate(bom_dataframe, component_id, df):
    # Find the FINAL_COMPONENT_UOM for the given COMPONENT_ID. the following to fetch informaiton for the root node
    final_component_uom = df.loc[df['OUTPUT_MATERIAL_ID'] == component_id, 'FINAL_OUTPUT_MATERIAL_UOM'].values[0]
    OUTPUT_MATERIAL_DESC = df.loc[df['OUTPUT_MATERIAL_ID'] == component_id, 'OUTPUT_MATERIAL_DESC'].values[0]
    OUTPUT_MATERIAL_TYPE = df.loc[df['OUTPUT_MATERIAL_ID'] == component_id, 'OUTPUT_MATERIAL_TYPE'].values[0]
    OUTPUT_MASS_CONV_FACTOR = df.loc[df['OUTPUT_MATERIAL_ID'] == component_id, 'OUTPUT_MASS_CONV_FACTOR'].values[0]
    OUTPUT_MASS_UNIT = df.loc[df['OUTPUT_MATERIAL_ID'] == component_id, 'OUTPUT_MASS_UNIT'].values[0]
    PLANT_ID = df.loc[df['OUTPUT_MATERIAL_ID'] == component_id, 'PLANT_ID'].values[0]
    
    # Extract component-specific totals into a dictionary
    result_dict = {
        row['COMPONENT_ID']: {
            'TOTAL_INPUT_QTY': row['TOTAL_INPUT_QTY'],
            'TOTAL_OUTPUT_QTY': row['TOTAL_OUTPUT_QTY']
        }
        for _, row in bom_dataframe.iterrows()
    }

    

    # Create a default row using the provided component_id and its FINAL_COMPONENT_UOM
    default_row = pd.DataFrame({
        'Level': [0],
        'COMPONENT_ID': [component_id],  # Use the input component_id instead of the hardcoded value
        'TOTAL_INPUT_QTY': [1],
        'FINAL_COMPONENT_UOM': [final_component_uom],  # Use the fetched FINAL_COMPONENT_UOM
       'COMPONENT_DESC': [OUTPUT_MATERIAL_DESC],
       'COMPONENT_MATERIAL_TYPE': [OUTPUT_MATERIAL_TYPE],
        'COMPONENT_MASS_CONV_FACTOR': [OUTPUT_MASS_CONV_FACTOR],
        'COMPONENT_MASS_UNIT':[OUTPUT_MASS_UNIT],
        'PLANT_ID':[PLANT_ID]
    })
    
    bom_dataframe = pd.concat([default_row, bom_dataframe], ignore_index=True)

    # Define the desired column order
    desired_column_order = ['Level', 'PLANT_ID', 'COMPONENT_ID', 'COMPONENT_DESC','COMPONENT_MATERIAL_TYPE', 'TOTAL_INPUT_QTY', 'FINAL_COMPONENT_UOM','COMPONENT_MASS_CONV_FACTOR','COMPONENT_MASS_UNIT']  # Example order

    bom_dataframe = bom_dataframe[desired_column_order]
    
    return bom_dataframe, result_dict


# scalling quantities

In [6]:
def scale_quantities(grouped_df, result_dict):
    # Dictionary to store scaled quantities
    scaled_quantity_dict = {}

    # Iterate through grouped_df to scale TOTAL_INPUT_QTY
    for index, row in grouped_df.iterrows():
        component_id = row['COMPONENT_ID']
        level = row['Level']
        
        if level == 0:
            # For top-level items, retain original TOTAL_INPUT_QTY
            scaled_quantity_dict[component_id] = row['TOTAL_INPUT_QTY']
        else:
            # Calculate scaled TOTAL_INPUT_QTY based on scaling_dict
            total_input_qty = result_dict[component_id]['TOTAL_INPUT_QTY']
            total_output_qty = result_dict[component_id]['TOTAL_OUTPUT_QTY']
            
            # Avoid division by zero
            if total_output_qty != 0:
                scaled_qty = total_input_qty / total_output_qty
            else:
                scaled_qty = 0  # Default to 0 if the output quantity is zero
            
            scaled_quantity_dict[component_id] = scaled_qty

    # Update grouped_df with scaled quantities
    grouped_df['TOTAL_INPUT_QTY'] = grouped_df['COMPONENT_ID'].map(scaled_quantity_dict)

    return grouped_df


In [7]:
def scale_input_quantities_by_level(grouped_df):
    # Dictionary to keep track of the last quantity seen at each level
    last_level_qty = {}

    # Iterate over each row in the grouped_df
    for idx, row in grouped_df.iterrows():
        level = row['Level']
        item_id = row['COMPONENT_ID']
        
        if level == 0:
            # Top-level item, keep its quantity as it is
            last_level_qty[level] = row['TOTAL_INPUT_QTY']
        else:
            # Parent is the item at the previous level (level - 1)
            parent_qty = last_level_qty[level - 1]
            
            # Multiply the current item’s quantity by the parent's quantity
            row['TOTAL_INPUT_QTY'] *= parent_qty
            
            # Update the DataFrame with the new quantity
            grouped_df.at[idx, 'TOTAL_INPUT_QTY'] = row['TOTAL_INPUT_QTY']
            
            # Update last_level_qty for this level
            last_level_qty[level] = row['TOTAL_INPUT_QTY']

    return grouped_df

In [8]:
def add_leaf_status(grouped_df):
    # Initialize the 'is_leaf' column to 'Yes' for all items
    grouped_df['is_leaf'] = 'Yes'

    # Iterate over the DataFrame rows to check each item
    for i in range(len(grouped_df) - 1):
        # If the next item's level is greater than the current item's level,
        # then the current item has a child and is not a leaf
        if grouped_df.loc[i + 1, 'Level'] > grouped_df.loc[i, 'Level']:
            grouped_df.loc[i, 'is_leaf'] = 'No'

    return grouped_df


# Geography enrichment

In [9]:
def enrich_with_supplier_geo(plant, plant_loc, grouped_df):
    # Initialize new columns in grouped_df
    grouped_df['Supplier'] = None
    grouped_df['Geo_1'] = None

    # Loop through each row in grouped_df
    for idx, row in grouped_df.iterrows():
        # Check if the row is a non-leaf
        if row['is_leaf'] == 'No':
            # Find matching plant information
            plant_info = plant[plant['PLANT_ID'] == row['PLANT_ID']]
            
            # If a match is found, use the first match
            if not plant_info.empty:
                plant_info = plant_info.iloc[0]  # Get the first match
                
                # Fill Supplier from DESCRIPTION
                grouped_df.at[idx, 'Supplier'] = plant_info['DESCRIPTION']
                
                # Check for Geo_1
                latitude = plant_info['LATITUDE']
                longitude = plant_info['LONGITUDE']
                if (latitude != 0) and (longitude != 0):
                    grouped_df.at[idx, 'Geo_1'] = f"{latitude}, {longitude}"
                else:
                    # If Geo_1 is not valid, construct Geo_1 from address components
                    address_parts = [
                        plant_info['STREET_ADDRESS'],
                        plant_info['CITY'],
                        plant_info['STATE'],
                        plant_info['COUNTRY']
                    ]
                    address_parts = [part for part in address_parts if part not in [None, 'unknown']]
                    grouped_df.at[idx, 'Geo_1'] = ', '.join(address_parts)

        # Check if the row is a leaf
        elif row['is_leaf'] == 'Yes':
            # Find matching plant_loc information
            loc_info = plant_loc[
                (plant_loc['MATERIAL_ID'] == row['COMPONENT_ID']) &
                (plant_loc['PLANT_ID'] == row['PLANT_ID'])
            ]
            
            # If a match is found, use the first match
            if not loc_info.empty:
                loc_info = loc_info.iloc[0]  # Get the first match
                
                # Fill Supplier and Geo_1 for leaf nodes
                grouped_df.at[idx, 'Supplier'] = loc_info['SUPPLIER_ID']
                grouped_df.at[idx, 'Geo_1'] = loc_info['SUPPLIER_COUNTRY_CODE']

    # Return the final enriched DataFrame
    return grouped_df


# Composition enrichment

In [10]:
def add_cdms_matches_to_grouped_df(grouped_df, CDMS_DATA):
    # Function to find matches in CDMS_DATA based on STOCK_NUMBER or SAP_ID
    def get_matches(component_id):
        # Find matches in CDMS_DATA where either STOCK_NUMBER or SAP_ID matches the COMPONENT_ID
        matches = CDMS_DATA[(CDMS_DATA['STOCK_NUMBER'] == component_id) | (CDMS_DATA['SAP_ID'] == component_id)]
        # Return the matching rows as dictionaries for COMPONENT_NAME, CAS_, and MAX_VALUE
        return matches[['COMPONENT_NAME', 'CAS_', 'MAX_VALUE']].to_dict(orient='records')

    # Populate the 'temp' column with matches based on the logic
    grouped_df['temp'] = grouped_df.apply(
        lambda row: get_matches(row['COMPONENT_ID']) if row['is_leaf'] == 'Yes' else None,
        axis=1
    )

    # Return the updated grouped_df
    return grouped_df


In [11]:
import pandas as pd

def expand_grouped_df(grouped_df):
    # Initialize an empty list to hold the final expanded rows
    expanded_rows = []

    # Iterate through each row in grouped_df
    for _, row in grouped_df.iterrows():
        # Append the original row to expanded_rows as-is
        expanded_rows.append(row.to_dict())

        # Check if 'temp' column has non-empty matches
        if row['temp'] is not None and len(row['temp']) > 0:
            # For each match in 'temp', create a new row with only the required columns filled
            for match in row['temp']:
                # Only proceed if CAS_ is valid (not NaN or empty)
                if pd.notna(match.get('CAS_')): # removing the part that checks for empty CAS == and match.get('CAS_'): ==  # Safely access 'CAS_' with get()
                    # Start a new row with all columns set to None initially
                    new_row = {col: None for col in grouped_df.columns}

                    # Populate only the specified columns
                    new_row.update({
                        'COMPONENT_ID': 'chemical',  # You can retain row['COMPONENT_ID'] if needed for reference
                        'is_leaf': 'No',  # Indicate this is an expanded row
                        'Amount': match.get('MAX_VALUE', 0) * row['TOTAL_INPUT_QTY'],  # MAX_VALUE from match * amount of parent, default to 0
                        'Level': row['Level'] + 1,  # Increase Level by 1
                        'Name': match.get('COMPONENT_NAME', ''),  # COMPONENT_NAME from the match
                        'CAS_': match.get('CAS_'),  # CAS_ from the match
                        'Geo_1': row['Geo_1'],
                        'Max_proportion': match.get('MAX_VALUE', 0)  # Default to 0 if MAX_VALUE is missing
                    })

                    # Append the new row to expanded_rows
                    expanded_rows.append(new_row)
        else:
            # If 'temp' is None or empty, add a row with empty values but inherit key columns
            empty_row = {col: None for col in grouped_df.columns}
            
            # Set necessary columns to maintain structure
            empty_row.update({
                'COMPONENT_ID': 'chemical', # or use row['COMPONENT_ID'],
                'is_leaf': 'Yes',  # Could indicate this is an unexpanded row
                'Level': row['Level'],  # Keep the same level as the original row
                'Geo_1': row['Geo_1'],
                'Amount': None,
                'Max_proportion': None,
                'Name': None,
                'CAS_': None
            })
            
            # Append the empty row to expanded_rows
            expanded_rows.append(empty_row)

    # Convert expanded_rows to a DataFrame
    final_df = pd.DataFrame(expanded_rows)

    # Drop 'temp' column if it exists
    final_df = final_df.drop(columns=['temp'], errors='ignore')

    # Return the final dataframe
    return final_df


In [12]:
import pandas as pd

def final_rescaling(final_df):
    # Step 1: Identify the start of each new chemical group
    final_df['Is_New_Group'] = (
        (final_df['COMPONENT_ID'] != 'chemical') & 
        (final_df['COMPONENT_ID'].shift(-1) == 'chemical')
    ).shift(fill_value=False).cumsum()

    # Step 2: Filter to include only rows with chemicals
    chemical_subelements = final_df[final_df['COMPONENT_ID'] == 'chemical']

    # Step 3: Group by 'Is_New_Group' and calculate the sum of 'Max_proportion' for each group
    chemical_sums = chemical_subelements.groupby('Is_New_Group')['Max_proportion'].transform('sum')

    # Step 4: Calculate 'TOTAL_INPUT_QTY' for each chemical based on 'Amount' and the group's 'Max_proportion' sum
    final_df.loc[final_df['COMPONENT_ID'] == 'chemical', 'TOTAL_INPUT_QTY'] = (
        chemical_subelements['Amount'] / chemical_sums
    )
    final_df.drop(final_df[(final_df['COMPONENT_ID'] == 'chemical') & (final_df['Name'].isna()) & (final_df['CAS_'].isna())].index, inplace=True)

    final_df.drop(columns=['is_leaf', 'Is_New_Group','Amount'], inplace=True)
    # Return the final DataFrame with rescaling applied
    return final_df


In [13]:
import pandas as pd

def extra_bom_modifications(df):
    """
    Perform specific modifications to a dataframe for rows where COMPONENT_ID == 'chemical'.
    
    Args:
        df (pd.DataFrame): Input dataframe.
        
    Returns:
        pd.DataFrame: Modified dataframe with changes applied.
    """
    # Create a copy to avoid modifying the original dataframe
    modified_df = df.copy()
    
    # Perform modifications where COMPONENT_ID == 'chemical'
    mask = modified_df['COMPONENT_ID'] == 'chemical'
    modified_df.loc[mask, 'TOTAL_INPUT_QTY'] *= 0.453592  # Convert from LB to KG
    modified_df.loc[mask, 'FINAL_COMPONENT_UOM'] = 'KG'   # Update unit to KG
    modified_df.loc[mask, 'COMPONENT_ID'] = modified_df.loc[mask, 'Name']  # Replace 'chemical' with actual name

    # Reset the index
    modified_df.reset_index(drop=True, inplace=True)
    return modified_df


In [14]:
import numpy as np
def extra_bom_modifications_2(final_df, plant):
    # Create a copy of the plant DataFrame to work on
    plant_copy = plant.copy()

    # Replace 'unknown' with NaN in specified columns
    columns_to_replace = ['STREET_ADDRESS', 'CITY', 'STATE', 'REGION', 'POSTAL_CODE', 'COUNTRY']
    plant_copy[columns_to_replace] = plant_copy[columns_to_replace].replace('Unknown', np.nan)

    # Function to create the location column
    def generate_location(row):
        if row['LATITUDE'] != 0 and row['LONGITUDE'] != 0:
            return f"{row['LATITUDE']}, {row['LONGITUDE']}"
        else:
            address_parts = [
                row.get('STREET_ADDRESS'),
                row.get('CITY'),
                row.get('STATE'),
                row.get('REGION'),
                row.get('POSTAL_CODE'),
                row.get('COUNTRY')
            ]
            # Filter out None or NaN values and join the address parts
            return ', '.join(filter(pd.notna, address_parts))

    # Apply the function to create the location column
    plant_copy['geo'] = plant_copy.apply(generate_location, axis=1)
    plant_copy.rename(columns={'DESCRIPTION': 'SUPPLIER'}, inplace=True)

    # Filter final_df for rows where FINAL_COMPONENT_UOM == 'HR'
    filtered_final_df = final_df[final_df['FINAL_COMPONENT_UOM'] == 'HR'].reset_index()

    # Merge filtered_final_df with plant_copy on PLANT_ID
    merged_df = filtered_final_df.merge(plant_copy, on='PLANT_ID', how='left')

    # Update Supplier and Geo_1 columns in the original final_df using the original indices
    final_df.loc[final_df['FINAL_COMPONENT_UOM'] == 'HR', 'Supplier'] = merged_df['SUPPLIER'].values
    final_df.loc[final_df['FINAL_COMPONENT_UOM'] == 'HR', 'Geo_1'] = merged_df['geo'].values

    # Initialize the 'is_leaf' column to 'Yes' for all items
    final_df['is_leaf'] = 'Yes'

    # Iterate over the DataFrame rows to check each item
    for i in range(len(final_df) - 1):
        # If the next item's level is greater than the current item's level,
        # then the current item has a child and is not a leaf
        if final_df.loc[i + 1, 'Level'] > final_df.loc[i, 'Level']:
            final_df.loc[i, 'is_leaf'] = 'No'

    # Iterate through rows in final_df
    for index, row in final_df.iterrows():
        if row['is_leaf'] == 'No' and row['Level'] != 0:
            # Find the next leaf item
            next_leaf = final_df.loc[(final_df.index > index) & (final_df['is_leaf'] == 'Yes')].head(1)
            if not next_leaf.empty:
                next_leaf_plant_id = next_leaf.iloc[0]['PLANT_ID']
                # If PLANT_IDs differ, fetch the SUPPLIER and geo from plant_copy DataFrame
                if row['PLANT_ID'] != next_leaf_plant_id:
                    description = plant_copy.loc[plant_copy['PLANT_ID'] == next_leaf_plant_id, 'SUPPLIER'].values
                    geo_1 = plant_copy.loc[plant_copy['PLANT_ID'] == next_leaf_plant_id, 'geo'].values
                    if description.size > 0:
                        final_df.at[index, 'Supplier'] = description[0]
                    if geo_1.size > 0:
                        final_df.at[index, 'Geo_1'] = geo_1[0]

    # Drop the 'is_leaf' column
    final_df.drop(columns=['is_leaf'], inplace=True)

    return final_df


# Running the Script

In [15]:
df, plant, plant_loc, CDMS_DATA = load_and_prepare_data()

In [16]:
adj_full = create_adj_list(df)


Elapsed time: 3.7727978229522705 seconds


In [17]:
def main(component_id):
    # Step 1: Build the BOM dataframe
    bom_dataframe = build_bom_dataframe(component_id, adj_full, df)
    # Step 2: Group and aggregate
    bom_dataframe, result_dict = group_and_aggregate(bom_dataframe, component_id, df)
    
    # Step 3: Scale quantities
    bom_dataframe = scale_quantities(bom_dataframe, result_dict)
    
    # Step 4: Scale input quantities by level
    bom_dataframe = scale_input_quantities_by_level(bom_dataframe)
    
    # Step 5: Add leaf status
    bom_dataframe = add_leaf_status(bom_dataframe)
    # Step 6: Enrich with supplier geography
    bom_dataframe = enrich_with_supplier_geo(plant, plant_loc, bom_dataframe)
    # Step 7: Add CDMS matches
    bom_dataframe = add_cdms_matches_to_grouped_df(bom_dataframe, CDMS_DATA)
    # Step 8: Expand grouped dataframe
    bom_dataframe = expand_grouped_df(bom_dataframe)
    # Step 9: Final rescaling
    bom_dataframe = final_rescaling(bom_dataframe)
    
    # step 10: extra BOM modifications
    
    bom_dataframe = extra_bom_modifications(bom_dataframe)
    bom_dataframe = extra_bom_modifications_2(bom_dataframe,plant)
    return bom_dataframe


In [18]:
len(df)

1394053

In [19]:
bom_dataframe = main('7000121378')
bom_dataframe

,Level,PLANT_ID,COMPONENT_ID,COMPONENT_DESC,COMPONENT_MATERIAL_TYPE,TOTAL_INPUT_QTY,FINAL_COMPONENT_UOM,COMPONENT_MASS_CONV_FACTOR,COMPONENT_MASS_UNIT,Supplier,Geo_1,Max_proportion,Name,CAS_
0,0,1012,7000121378,49 CL 255GAL DR,FERT,1.000000,DR,955.587,KG,3M Cordova,"41.7539, -90.2889",NaN,NaN,NaN
1,1,1012,Machine-79452N,MACHINE,None,0.001526,HR,NaN,None,3M Cordova,"41.7539, -90.2889",NaN,NaN,NaN
2,1,1012,3010214181,"CONTAINER, INTERMEDIATE BULK 275 GAL",VERP,0.800000,EA,57152.600,G,1722699,US,NaN,NaN,NaN
3,1,1012,4100059230,PSA00871 WIP SS LATEX FASTBOND INPUT,HALB,952.716541,KG,1.000,KG,3M Cordova,"41.7539, -90.2889",NaN,NaN,NaN
4,2,1012,Machine-79452N,MACHINE,None,1.453862,HR,NaN,None,3M Cordova,"41.7539, -90.2889",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,4,None,Polyglycol Ether,None,None,0.001781,KG,NaN,None,None,US,1.0,Polyglycol Ether,
1700,4,None,Diethanolamine,None,None,0.000891,KG,NaN,None,None,US,0.5,Diethanolamine,111-42-2
1701,4,None,Ethanolamine,None,None,0.000178,KG,NaN,None,None,US,0.1,Ethanolamine,141-43-5
1702,4,None,Triethanolamine,None,None,0.178115,KG,NaN,None,None,US,100.0,Triethanolamine,102-71-6


In [20]:
bom_dataframe.to_excel("7000121378_with_cycle_example.xlsx",index=False)

In [54]:
bom_dataframe[bom_dataframe['COMPONENT_ID'] == '11002103734']

,Level,PLANT_ID,COMPONENT_ID,COMPONENT_DESC,COMPONENT_MATERIAL_TYPE,TOTAL_INPUT_QTY,FINAL_COMPONENT_UOM,COMPONENT_MASS_CONV_FACTOR,COMPONENT_MASS_UNIT,Supplier,Geo_1,Max_proportion,Name,CAS_
318,6,1000,11002103734,VULCANIZED FIBER 0.0335 (DYNOS) CHOCOLATE BROW...,ROH,0.277085,YD,NaN,None,1723730,DE,NaN,NaN,NaN
637,6,1000,11002103734,VULCANIZED FIBER 0.0335 (DYNOS) CHOCOLATE BROW...,ROH,0.000580,YD,NaN,None,1723730,DE,NaN,NaN,NaN
957,7,1000,11002103734,VULCANIZED FIBER 0.0335 (DYNOS) CHOCOLATE BROW...,ROH,0.000196,YD,NaN,None,1723730,DE,NaN,NaN,NaN


In [56]:
#bom_dataframe.to_excel('7000045160.xlsx',index=False)

In [ ]:
bom_dataframe[(bom_dataframe['COMPONENT_ID'] == '1000002305')]

In [17]:
import pandas as pd
without_cycle = pd.read_excel('without_cycle_2023_updated_2.xlsx')
without_cycle

,Material ID,Has Cycle
0,7000028199,False
1,7012512614,False
2,7100050520,False
3,7100182457,False
4,7000045159,False
...,...,...
480,7100064741,False
481,7100204776,False
482,7100219533,False
483,7010350043,False


In [18]:
without_cycle = without_cycle.drop_duplicates(subset='Material ID', keep='first')

In [19]:
without_cycle['Material ID'].duplicated().sum()

0

In [20]:
#without_cycle['Material ID'] = without_cycle['Material ID'].astype(str)
without_cycle

,Material ID,Has Cycle
0,7000028199,False
1,7012512614,False
2,7100050520,False
3,7100182457,False
4,7000045159,False
...,...,...
473,7100032448,False
478,7000034390,False
479,7010347546,False
480,7100064741,False


In [21]:
import pandas as pd
from tqdm import tqdm

# Initialize an empty list to store the DataFrames temporarily
chunk_size = 5  # Set chunk size to 10 rows (suitable for your processing time)
results_list = []

# Open a file to save the final result incrementally
#output_file = 'BOM_lists/3M_BOM_list_2023_updated_3.csv'
output_file = 'BOM_lists/3M_BOM_list_2023_updated_4.csv'

# Wrap the loop with tqdm to show progress bar
for i, material_id in enumerate(tqdm(without_cycle['Material ID'], desc="Creating BOMs", unit="BOM")):
    result_df = main(material_id)  # Apply the given main() function to each material ID
    results_list.append(result_df)  # Append the returned DataFrame to the results list
    
    # If we reach the chunk size, save and reset the list to free memory
    if (i + 1) % chunk_size == 0:
        chunk_df = pd.concat(results_list, ignore_index=True)
        chunk_df.to_csv(output_file, mode='a', header=not bool(i), index=False)  # Append to CSV file
        results_list.clear()  # Clear the list to free memory 

# At the end, save any remaining results
if results_list:
    chunk_df = pd.concat(results_list, ignore_index=True)
    chunk_df.to_csv(output_file, mode='a', header=not bool(i), index=False)

# Print or return the final concatenated DataFrame (optional)
final_result_df = pd.read_csv(output_file)  # Read back from the file if needed
print(final_result_df)


Creating BOMs: 100%|██████████████████████████████████████████████████████████████| 355/355 [4:25:13<00:00, 44.83s/BOM]


        0  1000   7000028199      987C 60+ 125 X 22MM SLOT 25/100 DC  FERT  \
0       1  1000  34870543674                            BAG, PLASTIC  VERP   
1       1  1000  34871966114                           INSERT, PAPER  VERP   
2       1  1000  34871626908          14" CF WEB WIDTH, 3 MIL OLEFIN  VERP   
3       1  1000  44000947885  INPR FIB X5 60+ 3M987C 125 X 22MM SLOT  HALB   
4       2  1000   4010038414               987C 60+ X5 FLEX 46.25 IN  HALB   
...    ..   ...          ...                                     ...   ...   
168799  2   NaN    Magnesium                                     NaN   NaN   
168800  2   NaN       Copper                                     NaN   NaN   
168801  2   NaN         Iron                                     NaN   NaN   
168802  2   NaN    Manganese                                     NaN   NaN   
168803  1  1249    VENDORVAL                          OUTSOURCED MFG   NaN   

             1.0  YD2  1118.2450532    G  3M Alexandria  45.869

In [22]:
# Reading a CSV file without a header
df_final_boms = pd.read_csv('BOM_lists/3M_BOM_list_2023_updated_4.csv', header=None)

In [24]:
#the following code removes non-printable cahracters from the BOMs if neeced so that it can be converted to
#excel
import pandas as pd
import re

# Function to remove non-printable characters
def clean_text(text):
    if isinstance(text, str):
        # Use regex to remove non-printable characters
        return re.sub(r'[^\x20-\x7E]', '', text)  # Keeps printable ASCII characters
    return text

# Apply the cleaning function to the entire DataFrame
df_final_boms = df_final_boms.apply(lambda col: col.map(clean_text))



In [25]:

# New column names
new_column_names = [
    "Level","PLANT_ID", "COMPONENT_ID", 'COMPONENT_DESC','COMPONENT_MATERIAL_TYPE' , "TOTAL_INPUT_QTY", "FINAL_COMPONENT_UOM", 'COMPONENT_MASS_CONV_FACTOR','COMPONENT_MASS_UNIT', 
     "Supplier", "Geo_1", "Max_proportion", 
    "Name", "CAS_"
]

# Assign the new column names
df_final_boms.columns = new_column_names
df_final_boms
df_final_boms.to_excel('BOM_lists/3M_BOM_list_2023_updated_4.xlsx',index=False)